In [ ]:
from langchain_qdrant import QdrantVectorStore, FastEmbedSparse, SparseEmbeddings
from qdrant_client import QdrantClient
from dotenv import load_dotenv
import os

from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_tavily import TavilySearch

from langchain_core.tools import Tool
from dataclasses import dataclass

from pydantic import BaseModel
from typing import List, Annotated
from langchain_core.messages import AnyMessage, AIMessage, HumanMessage
from langgraph.graph.message import add_messages

In [18]:
client = QdrantClient(
    url=os.getenv('QDRANT_API_URL'),
    api_key= os.getenv('QDRANT_API_KEY')
)

In [19]:
dense_embedding = HuggingFaceEmbeddings(model = 'sentence-transformers/all-MiniLM-L12-v2')
sparse_embedding = FastEmbedSparse(model_name='Qdrant/bm25')

In [20]:
qdrantdb = QdrantVectorStore(
    client=client,
    collection_name= os.getenv('COLLECTIONNAME'),
    embedding= dense_embedding,
    sparse_embedding= sparse_embedding
)

In [21]:
qdrantdb_retriver = qdrantdb.as_retriever(
    search_type = 'mmr',
    search_kwarg = {
        "k":3
    }
)
qdrantdb_retriver.invoke("Create Test Measure for Same Period Last Year")

[Document(metadata={'created_date': '2025-11-13 17:16:45', 'author': '', 'tags': 'DAX Questions.pdf', 'producer': 'Skia/PDF m144 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '..\\data\\PowerBI\\DAX Questions.pdf', 'file_path': '..\\data\\PowerBI\\DAX Questions.pdf', 'total_pages': 12, 'format': 'PDF 1.4', 'title': '31. What is DAX, and why is it important in Power...', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 6, '_id': '2dc911b3-1ea7-4b82-88b7-dfc6458f3364', '_collection_name': 'PowerBIChat'}, page_content="Write a DAX measure to calculate year-to-date (YTD) sales. Using the TOTALYTD function (which wraps CALCULATE and DATESYTD): Code snippet Sales YTD = TOTALYTD(SUM('Sales'[Amount]), 'Date'[Date])\u200b 42. Write a DAX measure to calculate the difference between current year sales and previous year sales. Code snippet"),
 Document(metadata={'created_date': '2025-11-13 17:16:45', 'author': '', 'tags':

In [22]:
def getPowerBIQuestionAnswers(query : str) -> str:

    docs = qdrantdb_retriver.invoke(query)
    return '\n\n'.join([doc.page_content for doc in docs])

powerbi_qns_tool = Tool(
    name = "getPowerBIQuestionAnswers",
    description= "get PowerBI Question Answers from qdrant DB",
    func= getPowerBIQuestionAnswers
)

In [23]:
wiki_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results= 5, doc_content_chars_max=2000))
tivily_tool = TavilySearch(
    max_results=5,
    topic="general",
)


In [ ]:
tools = [wiki_tool, powerbi_qns_tool, tivily_tool]

In [ ]:
@dataclass
class PBIBot(BaseModel):
    messages: Annotated[List[AnyMessage], add_messages]

In [ ]:
def agent(state: PBIBot):
    """ 
    Represents the core agent node in a LangGraph workflow.

    This function is responsible for determining the next course of action based on the
    current conversation history (state). It decides whether to generate a final response
    or use one of the provided tools (which can include retrieval tools or general tools).

    Args:
        state: The current conversation history, represented as a list of LangChain
               `BaseMessage` objects. This is the input to the agent's decision-making
               process.
    
    Returns:
        A list containing the new message(s) generated by the agent or the results
        from the executed tools. The state update logic in LangGraph handles appending
        these new messages to the overall conversation history.
    """